### Tutorial 05: Advanced Query Techniques of CASE and Subquery

In this tutorial, we will explore two advanced SQL query techniques: CASE statements and subqueries. These powerful tools allow you to write more flexible and dynamic SQL queries, which can be incredibly useful when working with complex databases.

**CASE Statements**: Conditional Logic in Queries
The CASE statement in SQL allows you to add conditional logic to your queries. It works similarly to an if-else or switch statement in programming.

**Example 1**: Classifying Participants by Age Group
Let’s say we want to classify participants into different age groups such as:

- Youth: Age <= 24
- Young Adult: Age between 25 and 39
- ADult: Age > 40
---
**Subqueries**: Nesting Queries for Flexibility
A subquery is a query within another query. Subqueries can be used in SELECT, INSERT, UPDATE, or DELETE statements, and they allow you to perform operations like filtering, computing aggregates, or updating values based on the results of another query.

**Example 2**: Using a Subquery in a `SELECT` Clause
Let's say we want to know the average age of all participants and display it next to each participant’s information.

**Example 3**: Using a Subquery in a `WHERE` Clause
Now, suppose we want to select participants whose age is greater than the average age of all participants. This requires a subquery in the `WHERE` clause.

---

**Combining CASE and Subqueries**
You can combine both techniques for even more powerful and dynamic queries. 
Display the participant’s age group along with a message indicating whether they are above or below average age.
**Example 4**, 

---

In [ ]:
import sqlite3
import pandas as pd

db_path = './database/mmdt.db3'

In [ ]:
update_query = """
                UPDATE 
                    participants 
                SET BOD = (SELECT substr(BOD, 7, 4) FROM bhutan) 
                WHERE BOD is NULL;
            
                """
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(update_query)
conn.commit()
conn.close()

In [ ]:
update_query = """
                UPDATE 
                    participants 
                SET BOD = (SELECT AVG(BOD) FROM participants) 
                WHERE BOD >=2014;
            
                """
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(update_query)
conn.commit()
conn.close()

In [ ]:
query = """
        SELECT
            ID, 
            CASE 
                WHEN 2024-BOD < 25 THEN 'Youth'
                WHEN 2024-BOD BETWEEN 25 and 39 THEN 'Young Adult'
                ELSE 'Adult'
            END AS age_category
            FROM participants;
        """

df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df

In [ ]:
query = """
        SELECT 
            ID, 2024-BOD as age,
            (SELECT ROUND(AVG(2024-BOD),2) FROM participants) as average_age
            FROM participants;
        """
df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df

In [ ]:
query = """
        SELECT 
            ID, 2024-BOD as age            
            FROM participants
            WHERE 
            (2024-BOD)> (SELECT AVG(2024-BOD) FROM participants);
        """


In [ ]:
query = """
        WITH ageData AS(
            SELECT
                ID, 
                (2024-BOD) as age, 
                CASE 
                    WHEN 2024-BOD < 25 THEN 'Youth'
                    WHEN 2024-BOD BETWEEN 25 and 39 THEN 'Young Adult'
                    ELSE 'Adult'
                END AS age_category
                FROM participants
            ),
        avgAge AS(SELECT AVG(2024-BOD) AS avg_age FROM participants)
        SELECT
            a.ID, a.age,
            a.age_category,
            CASE
                WHEN a.age> ag.avg_age THEN 'Above Average'
                ELSE 'Below Average'
            END as age_message
        FROM ageData AS a
        INNER JOIN avgAge AS ag;   
        """
df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df